In [1]:
import csv
import re
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
import tabula
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from fuzzywuzzy import process
import geopandas as gpd
import numpy as np

## 0. Leitura dos Ficheiros

In [2]:
# Guardar todos os outputs numa única pasta
# output_folder = Path.cwd() / "output"
# output_folder.mkdir(parents=True, exist_ok=True)

# Exemplo: mapa.save(output_folder / 'mapa_de_calor_portugal_Danielle.html')

In [3]:
p = Path.cwd()
data_folder = p.parent.parent / "data"
shape_folder = p.parent.parent / "shape"

## 1. Incêndios

In [4]:
file_path = data_folder / "Dados Incêndios Urbanos por Freguesia 2014-2021 ISCTE.xlsx"
Fogo = pd.read_excel(file_path)
Fogo

,Ano,Mês,Distrito,CIM,DICO,Concelho,DICOFRE,Freguesia,CodNatureza,Natureza,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10119,Valongo do Vouga,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,61.0
1,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10121,Águeda e Borralha,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,27.0
2,2014,1.0,Aveiro,Região de Aveiro,102.0,Albergaria-a-Velha,10209,Albergaria-a-Velha e Valmaior,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,4.0,259.0
3,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10312,Vila Nova de Monsarros,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,48.0
4,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10313,Vilarinho do Bairro,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44439,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182321,Ranhados,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,3.0,110.0
44440,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182338,Faíl e Vila Chã de Sá,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,68.0
44441,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182341,Viseu,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,3.0,7.0,207.0
44442,2021,12.0,Viseu,Viseu Dão Lafões,1824.0,Vouzela,182414,Fataunços e Figueiredo Das Donas,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,140.0


In [5]:
Fogo["Natureza"] = Fogo["Natureza"].str.split('/').str[-1].str.strip()
Fogo.dropna(subset=["Concelho"], inplace=True)
Fogo

,Ano,Mês,Distrito,CIM,DICO,Concelho,DICOFRE,Freguesia,CodNatureza,Natureza,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10119,Valongo do Vouga,2101.0,Habitacional,1.0,1.0,61.0
1,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10121,Águeda e Borralha,2101.0,Habitacional,1.0,1.0,27.0
2,2014,1.0,Aveiro,Região de Aveiro,102.0,Albergaria-a-Velha,10209,Albergaria-a-Velha e Valmaior,2101.0,Habitacional,1.0,4.0,259.0
3,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10312,Vila Nova de Monsarros,2101.0,Habitacional,1.0,1.0,48.0
4,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10313,Vilarinho do Bairro,2101.0,Habitacional,1.0,2.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44438,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182301,Abraveses,2101.0,Habitacional,3.0,7.0,171.0
44439,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182321,Ranhados,2101.0,Habitacional,1.0,3.0,110.0
44440,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182338,Faíl e Vila Chã de Sá,2101.0,Habitacional,1.0,2.0,68.0
44441,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182341,Viseu,2101.0,Habitacional,3.0,7.0,207.0


In [6]:
pivot_table = Fogo.pivot_table(Fogo, 
                             index=['Ano', 'Mês', 'Concelho'],
                             columns='Natureza',               
                             aggfunc='size',                   
                             fill_value=0)                     

pivot_table

Natureza                      Bibliotecas e Arquivos  Desporto e Lazer  \
Ano  Mês  Concelho                                                       
2014 1.0  Abrantes                                 0                 0   
          Aguiar da Beira                          0                 0   
          Albergaria-a-Velha                       0                 0   
          Albufeira                                0                 0   
          Alcanena                                 0                 0   
...                                              ...               ...   
2021 12.0 Vouzela                                  0                 0   
          Águeda                                   0                 0   
          Évora                                    0                 0   
          Ílhavo                                   0                 0   
          Óbidos                                   0                 0   

Natureza                      Edifícios degradados ou devolutos  \
Ano  Mês  Concelho                                                
2014 1.0  Abrantes                                            0   
          Aguiar da Beira                                     0   
          Albergaria-a-Velha                                  0   
          Albufeira                                           0   
          Alcanena                                            0   
...                                                         ...   
2021 12.0 Vouzela                                             0   
          Águeda                                              0   
          Évora                                               0   
          Ílhavo                                              0   
          Óbidos                                              0   

Natureza                      Espectáculos e Reuniões Públicas  \
Ano  Mês  Concelho                                               
2014 1.0  Abrantes                                           0   
          Aguiar da Beira                                    0   
          Albergaria-a-Velha                                 0   
          Albufeira                                          0   
          Alcanena                                           0   
...                                                        ...   
2021 12.0 Vouzela                                            0   
          Águeda                                             0   
          Évora                                              0   
          Ílhavo                                             0   
          Óbidos                                             0   

Natureza                      Estacionamento de superfície  \
Ano  Mês  Concelho                                           
2014 1.0  Abrantes                                       0   
          Aguiar da Beira                                0   
          Albergaria-a-Velha                             0   
          Albufeira                                      0   
          Alcanena                                       0   
...                                                    ...   
2021 12.0 Vouzela                                        0   
          Águeda                                         0   
          Évora                                          0   
          Ílhavo                                         0   
          Óbidos                                         0   

Natureza                      Estacionamento em profundidade ou silo  \
Ano  Mês  Concelho                                                     
2014 1.0  Abrantes                                                 0   
          Aguiar da Beira                                          0   
          Albergaria-a-Velha                                       0   
          Albufeira                                                0   
          Alcanena                                                 

In [7]:
FogoPosto = Fogo.groupby(["Ano", "Mês", "Concelho"])[["Ocorrências", "MeiosTerrestres", "Duração Minutos"]].sum()
FogoPosto

Ocorrências  MeiosTerrestres  Duração Minutos
Ano  Mês  Concelho                                                         
2014 1.0  Abrantes                    1.0              1.0             44.0
          Aguiar da Beira             1.0              2.0             85.0
          Albergaria-a-Velha          1.0              4.0            259.0
          Albufeira                   1.0              3.0            107.0
          Alcanena                    3.0              4.0            133.0
...                                   ...              ...              ...
2021 12.0 Vouzela                     1.0              2.0            140.0
          Águeda                      1.0              1.0             39.0
          Évora                       2.0              7.0            327.0
          Ílhavo                      1.0              6.0            521.0
          Óbidos                      2.0              5.0            438.0

[16686 rows x 3 columns]

In [8]:
FogoFinal = pd.merge(pivot_table, FogoPosto, how='inner', on=['Ano', 'Mês', 'Concelho'])
FogoFinal

Bibliotecas e Arquivos  Desporto e Lazer  \
Ano  Mês  Concelho                                                       
2014 1.0  Abrantes                                 0                 0   
          Aguiar da Beira                          0                 0   
          Albergaria-a-Velha                       0                 0   
          Albufeira                                0                 0   
          Alcanena                                 0                 0   
...                                              ...               ...   
2021 12.0 Vouzela                                  0                 0   
          Águeda                                   0                 0   
          Évora                                    0                 0   
          Ílhavo                                   0                 0   
          Óbidos                                   0                 0   

                              Edifícios degradados ou devolutos  \
Ano  Mês  Concelho                                                
2014 1.0  Abrantes                                            0   
          Aguiar da Beira                                     0   
          Albergaria-a-Velha                                  0   
          Albufeira                                           0   
          Alcanena                                            0   
...                                                         ...   
2021 12.0 Vouzela                                             0   
          Águeda                                              0   
          Évora                                               0   
          Ílhavo                                              0   
          Óbidos                                              0   

                              Espectáculos e Reuniões Públicas  \
Ano  Mês  Concelho                                               
2014 1.0  Abrantes                                           0   
          Aguiar da Beira                                    0   
          Albergaria-a-Velha                                 0   
          Albufeira                                          0   
          Alcanena                                           0   
...                                                        ...   
2021 12.0 Vouzela                                            0   
          Águeda                                             0   
          Évora                                              0   
          Ílhavo                                             0   
          Óbidos                                             0   

                              Estacionamento de superfície  \
Ano  Mês  Concelho                                           
2014 1.0  Abrantes                                       0   
          Aguiar da Beira                                0   
          Albergaria-a-Velha                             0   
          Albufeira                                      0   
          Alcanena                                       0   
...                                                    ...   
2021 12.0 Vouzela                                        0   
          Águeda                                         0   
          Évora                                          0   
          Ílhavo                                         0   
          Óbidos                                         0   

                              Estacionamento em profundidade ou silo  \
Ano  Mês  Concelho                                                     
2014 1.0  Abrantes                                                 0   
          Aguiar da Beira                                          0   
          Albergaria-a-Velha                                       0   
          Albufeira                                                0   
          Alcanena                                                 0   
...                      

## 2. Densidade Populacional

In [9]:
file_path = data_folder / "PORDATA_Densidade-populacional.xlsx"
Denso = pd.read_excel(file_path, header=11, usecols=range(17))
Denso.drop("Âmbito Geográfico", axis=1, inplace=True)
Denso = Denso.dropna()
Denso.drop(334, inplace=True)
Denso

,Anos,2001,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos de Valdevez,55.2,52,51.5,50.6,49.7,49,48.4,47.9,47.4,47.1,46.8,46.6,46.5,46.4,46.2
1,Caminha,124.1,123.4,122.9,122.5,121.9,120.9,119.7,118.5,117.5,116.8,116.3,116.1,116.2,116.6,117.4
2,Melgaço,41.9,39.4,39,38.3,37.5,36.9,36.3,35.8,35.3,34.7,34.1,33.7,33.2,32.5,32.1
3,Monção,94.3,92,91.5,90.8,89.9,88.9,87.8,86.9,86.1,85.5,85,84.7,84.8,84.7,84.6
4,Paredes de Coura,69.1,67.2,66.9,66.1,65.3,64.8,64.2,63.8,63.4,63,62.8,62.8,63,63,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,193,204.3,204.6,204.6,203.5,201.2,198.1,195,192.9,192,192.3,193.7,194.8,195.3,196.2
304,Santa Cruz,448.9,505,520.4,528.3,527.2,523.7,519.1,514.5,511.5,510.4,510.7,512.3,518,524.7,531.2
305,Santana,91.2,83.1,81.6,80.1,78.5,77,75.4,73.8,72.4,71.2,70.4,69.9,69.4,68.6,68
306,São Vicente,76.5,74.1,73.2,71.7,70,68.5,66.6,65,63.8,62.8,62.1,61.7,61.8,61.8,61.7


## 3. População Residente

In [10]:
file_path = data_folder / "PORDATA_NUTS2013_Total.xlsx"
Viver = pd.read_excel(file_path, header=11, usecols=range(25))
Viver.drop("Âmbito Geográfico", axis=1, inplace=True)
Viver = Viver.dropna()
Viver.drop(333, inplace=True)
Viver

,Anos,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos de Valdevez,0,24686,24603,24448,24259,24062,23854,23692,23519,...,21944,21664,21420,21227,21076,20926,20844,20826,20754,20693
1,Caminha,0,17055,17100,17088,17051,17013,16986,16959,16892,...,16500,16345,16177,16045,15940,15873,15848,15863,15917,16033
2,Melgaço,0,9971,9949,9889,9796,9715,9630,9538,9465,...,8785,8652,8531,8418,8279,8136,8029,7901,7755,7648
3,Monção,0,19925,19950,19919,19843,19773,19698,19615,19536,...,18782,18556,18361,18199,18063,17953,17890,17909,17898,17868
4,Paredes de Coura,0,9565,9556,9509,9459,9429,9404,9368,9321,...,8952,8869,8811,8761,8706,8677,8685,8711,8703,8676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,12523,12683,12827,12971,13087,13165,13247,13316,...,13158,12969,12765,12625,12570,12585,12679,12749,12788,12841
304,Santa Cruz,0,30301,31734,33120,34466,35805,37154,38512,39847,...,42685,42307,41932,41690,41602,41625,41753,42115,42666,43190
305,Santana,0,8750,8685,8605,8511,8395,8276,8169,8062,...,7357,7204,7049,6911,6800,6722,6677,6624,6554,6490
306,São Vicente,0,6179,6174,6138,6078,6020,5979,5946,5904,...,5398,5255,5124,5027,4949,4897,4867,4872,4875,4861


## 4. Setor de Atividade

### 4.1. Primário

In [11]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkP = pd.read_excel(file_path, header=11, usecols=[0,1,6,7])
WorkP.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkP = WorkP.dropna()
WorkP.drop(333, inplace=True)
WorkP

,Anos,2011,2021
0,Arcos de Valdevez,514,374
1,Caminha,316,213
2,Melgaço,258,193
3,Monção,377,432
4,Paredes de Coura,185,151
...,...,...,...
303,Ribeira Brava,156,119
304,Santa Cruz,439,318
305,Santana,410,229
306,São Vicente,100,105


### 4.2. Secundário

In [12]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkS = pd.read_excel(file_path, header=11, usecols=[0,1,12,13])
WorkS.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkS = WorkS.dropna()
WorkS.drop(333, inplace=True)
WorkS.rename(columns={'2011.1': 2011}, inplace=True)
WorkS.rename(columns={'2021.1': 2021}, inplace=True)
WorkS

,Anos,2011,2021
0,Arcos de Valdevez,2369,2486
1,Caminha,1561,1634
2,Melgaço,595,579
3,Monção,2062,2058
4,Paredes de Coura,1253,1513
...,...,...,...
303,Ribeira Brava,1327,994
304,Santa Cruz,2777,2260
305,Santana,438,348
306,São Vicente,449,293


### 4.3. Setor Terciário

In [13]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkT = pd.read_excel(file_path, header=11, usecols=[0,1,18,19])
WorkT.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkT = WorkT.dropna()
WorkT.drop(333, inplace=True)
WorkT.rename(columns={'2011.2': 2011}, inplace=True)
WorkT.rename(columns={'2021.2': 2021}, inplace=True)
WorkT

,Anos,2011,2021
0,Arcos de Valdevez,4175,4141
1,Caminha,4386,4438
2,Melgaço,1767,1657
3,Monção,4077,4182
4,Paredes de Coura,1910,1750
...,...,...,...
303,Ribeira Brava,3454,3441
304,Santa Cruz,16902,16317
305,Santana,1855,1722
306,São Vicente,1351,1281


## 5. Indíce de Envelhecimento

In [14]:
file_path = data_folder / "IndiceEnvelhecimento.xlsx"
Velho = pd.read_excel(file_path, header=11, usecols=range(25))
Velho.drop("Âmbito Geográfico", axis=1, inplace=True)
Velho.dropna(inplace=True)
Velho.drop(333, inplace=True)
Velho

,Anos,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos de Valdevez,0,208.1,216.4,223.7,231.3,238.7,244,248.3,253.1,...,273.1,282.8,296,310.1,324.8,339.6,355.1,369.2,379.3,391.6
1,Caminha,0,138.1,141.1,145,147.3,149.9,153.1,156.4,161.3,...,203.4,209.8,218.9,229.2,239.7,247.6,253.3,258.9,265.8,272.3
2,Melgaço,0,297.4,305.3,311.1,321.1,332.3,348.2,367.2,376.7,...,393.4,410.7,433.9,451.1,473.6,501,526.9,548.2,563.3,577
3,Monção,0,209.7,217.6,222.3,227.2,234.8,242,246.3,246.3,...,277.2,284.3,292.3,301,310.2,318.2,324.1,332.2,343.5,351.5
4,Paredes de Coura,0,204.6,208.8,211.6,214.2,216.5,218.1,216.2,213.4,...,219,226,232.5,237.7,244.3,250.3,256.3,267.3,278.4,289.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,80.9,81,78.5,76.5,76.1,76,76,76.9,...,96.4,102.2,108,114.8,122.5,131.4,141.7,152,162.1,169.5
304,Santa Cruz,0,56.1,55.4,54.4,53.1,52.1,51.2,50.4,49.7,...,57.9,60.8,63.9,67.8,72.7,78.1,83.8,89.9,95.7,101.1
305,Santana,0,128.1,136.8,143.8,149,151.7,153.7,159.1,165.3,...,189.7,201.1,211.2,222.4,234.8,242.8,253,265.4,279.7,290.4
306,São Vicente,0,121.5,127,131.1,133.7,136.7,141.1,146,152,...,169.4,180.6,190.5,197.8,206,215.3,228.4,243.7,258.3,273.9


## 6. Zonas Rurais

In [15]:
file_path = data_folder / "ZonasCount.csv"
ZonasRurais = pd.read_csv(file_path)
ZonasRurais["Rural"] = ZonasRurais["Zona Rural"] / (ZonasRurais["Zona Urbana"] + ZonasRurais["Zona Rural"])
ZonasRurais

,Distrito,Concelho,Zona Rural,Zona Urbana,Rural
0,Aveiro,Albergaria-A-Velha,6,0,1.00
1,Aveiro,Anadia,10,0,1.00
2,Aveiro,Arouca,16,0,1.00
3,Aveiro,Aveiro,0,10,0.00
4,Aveiro,Castelo De Paiva,6,0,1.00
...,...,...,...,...,...
273,Évora,Reguengos De Monsaraz,4,0,1.00
274,Évora,Vendas Novas,2,0,1.00
275,Évora,Viana Do Alentejo,3,0,1.00
276,Évora,Vila Viçosa,4,0,1.00


## 7. Eventos Excecionais

In [30]:
file_path = data_folder / "eventos.parquet"
Quebras = pd.read_parquet(file_path)
Quebras.dropna(inplace=True)
Quebras.reset_index(drop = True, inplace=True)
Quebras['Ano'] = Quebras['data_do_incidente'].dt.year
Quebras['Mês'] = Quebras['data_do_incidente'].dt.month
Quebras['Dia'] = Quebras['data_do_incidente'].dt.day
Quebras.rename(columns={'concelho_origem_do_incidente': "Concelho"}, inplace=True)
Quebras["Concelho"] = Quebras["Concelho"].str.title()
Quebras

,codigo_do_relatorio,Concelho,data_do_incidente,causa_do_incidente,duracao_incidente_min,n_o_clientes_afectados,tiepi_mt_min,saifi_mt_number,saidi_mt_min,end_m_wh,saifi_bt_number,saidi_bt_min,nt,aprovado,Ano,Mês,Dia
0,EREDES_2023_NOV_TC_1,Barcelos,2023-11-01 22:30:00+00:00,VENTO INTENSIDADE EXCECIONAL,870.0,244990.0,1.807000,0.053000,2.822000,141.327000,0.049000,2.667000,AT,True,2023,11,1
1,EREDES_2023_OUT_T_6,Celorico De Basto,2023-10-09 16:04:00+00:00,VEÍCULOS,5.0,1416.0,0.001451,0.000284,0.001404,0.104147,0.000230,0.001139,MT,True,2023,10,9
2,EREDES_2023_OUT_T_10,Miranda Do Corvo,2023-10-09 09:43:00+00:00,ABATE DE ÁRVORES POR TERCEIROS,44.0,2693.0,0.002188,0.000105,0.004186,0.157058,0.000116,0.004531,MT,True,2023,10,9
3,EREDES_2023_OUT_T_12,Matosinhos,2023-10-13 15:25:00+00:00,INTERVENÇÕES NO SUBSOLO (ESCAVAÇÕES),31.0,3185.0,0.010268,0.000347,0.005273,0.736786,0.000517,0.004441,MT,True,2023,10,13
4,EREDES_2023_OUT_T_13,Santiago Do Cacém,2023-10-22 15:02:00+00:00,CORPOS ESTRANHOS NA REDE,130.0,238.0,0.004813,0.000200,0.020169,0.345438,0.000039,0.003832,MT,True,2023,10,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,EDPD_2014_JUN_T_065,Águeda,2014-06-10 00:00:00+00:00,ABATE DE ÁRVORES,93.0,61.0,0.012900,126.000000,0.009640,0.830000,10.000000,0.000880,MT,True,2014,6,10
5236,EDPD_2014_JUN_T_114,Viseu,2014-06-24 00:00:00+00:00,ABATE DE ÁRVORES,173.0,3652.0,0.014600,218.000000,0.025300,0.950000,259.000000,0.023300,MT,True,2014,6,24
5237,EDPD_2014_JUN_T_118,Figueira Da Foz,2014-06-07 00:00:00+00:00,ABATE DE ÁRVORES,168.0,2055.0,0.006640,276.000000,0.021000,0.430000,235.000000,0.017800,MT,True,2014,6,7
5238,EDPD_2014_JUN_T_124,Porto De Mós,2014-06-18 00:00:00+00:00,DESLIZAMENTO DE TERRAS,274.0,1834.0,0.010100,345.000000,0.027800,0.650000,302.000000,0.020900,MT,True,2014,6,18


In [31]:
Quebras = Quebras.merge(ZonasRurais[['Concelho', 'Rural']], on='Concelho', how='left')
Quebras

,codigo_do_relatorio,Concelho,data_do_incidente,causa_do_incidente,duracao_incidente_min,n_o_clientes_afectados,tiepi_mt_min,saifi_mt_number,saidi_mt_min,end_m_wh,saifi_bt_number,saidi_bt_min,nt,aprovado,Ano,Mês,Dia,Rural
0,EREDES_2023_NOV_TC_1,Barcelos,2023-11-01 22:30:00+00:00,VENTO INTENSIDADE EXCECIONAL,870.0,244990.0,1.807000,0.053000,2.822000,141.327000,0.049000,2.667000,AT,True,2023,11,1,0.934426
1,EREDES_2023_OUT_T_6,Celorico De Basto,2023-10-09 16:04:00+00:00,VEÍCULOS,5.0,1416.0,0.001451,0.000284,0.001404,0.104147,0.000230,0.001139,MT,True,2023,10,9,1.000000
2,EREDES_2023_OUT_T_10,Miranda Do Corvo,2023-10-09 09:43:00+00:00,ABATE DE ÁRVORES POR TERCEIROS,44.0,2693.0,0.002188,0.000105,0.004186,0.157058,0.000116,0.004531,MT,True,2023,10,9,1.000000
3,EREDES_2023_OUT_T_12,Matosinhos,2023-10-13 15:25:00+00:00,INTERVENÇÕES NO SUBSOLO (ESCAVAÇÕES),31.0,3185.0,0.010268,0.000347,0.005273,0.736786,0.000517,0.004441,MT,True,2023,10,13,0.000000
4,EREDES_2023_OUT_T_13,Santiago Do Cacém,2023-10-22 15:02:00+00:00,CORPOS ESTRANHOS NA REDE,130.0,238.0,0.004813,0.000200,0.020169,0.345438,0.000039,0.003832,MT,True,2023,10,22,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,EDPD_2014_JUN_T_065,Águeda,2014-06-10 00:00:00+00:00,ABATE DE ÁRVORES,93.0,61.0,0.012900,126.000000,0.009640,0.830000,10.000000,0.000880,MT,True,2014,6,10,1.000000
5236,EDPD_2014_JUN_T_114,Viseu,2014-06-24 00:00:00+00:00,ABATE DE ÁRVORES,173.0,3652.0,0.014600,218.000000,0.025300,0.950000,259.000000,0.023300,MT,True,2014,6,24,0.840000
5237,EDPD_2014_JUN_T_118,Figueira Da Foz,2014-06-07 00:00:00+00:00,ABATE DE ÁRVORES,168.0,2055.0,0.006640,276.000000,0.021000,0.430000,235.000000,0.017800,MT,True,2014,6,7,1.000000
5238,EDPD_2014_JUN_T_124,Porto De Mós,2014-06-18 00:00:00+00:00,DESLIZAMENTO DE TERRAS,274.0,1834.0,0.010100,345.000000,0.027800,0.650000,302.000000,0.020900,MT,True,2014,6,18,1.000000


In [33]:
Quebras[Quebras["Rural"].isnull()]

,codigo_do_relatorio,Concelho,data_do_incidente,causa_do_incidente,duracao_incidente_min,n_o_clientes_afectados,tiepi_mt_min,saifi_mt_number,saidi_mt_min,end_m_wh,saifi_bt_number,saidi_bt_min,nt,aprovado,Ano,Mês,Dia,Rural
262,EREDES_2022_DEZ_TC_2,Território De Portugal Continental,2022-12-11 00:00:00+00:00,VENTO INTENSIDADE EXCECIONAL,2280.0,387223.0,4.700,0.080000,6.840000,390.700000,0.07000,5.810000,AT,True,2022,12,11,NaN
617,EREDES_2021_OUT_TC_1,Território De Portugal Continental,2021-10-29 19:30:00+00:00,VENTO INTENSIDADE EXCECIONAL,1320.0,237413.0,3.081,0.079000,4.384000,200.406000,0.05400,2.909000,AT,True,2021,10,29,NaN
618,EREDES_2021_JUL_TC_1,Portugal Continental,2021-07-24 15:36:00+00:00,DESLASTRE DE CARGA AUTOMÁTICO,126.0,977394.0,7.454,0.186000,8.132000,506.029000,0.15600,6.894000,AT,True,2021,7,24,NaN
619,EDPD_2021_JAN_TC_1,Território De Portugal Continental,2021-01-21 22:30:00+00:00,VENTO INTENSIDADE EXCECIONAL,690.0,269663.0,1.730,0.060000,3.680000,126.030000,0.05000,2.540000,AT,True,2021,1,21,NaN
1467,EDPD_2020_MAR_TC_1,Território De Portugal Continental,2020-03-01 20:00:00+00:00,VENTO INTENSIDADE EXCECIONAL,840.0,266104.0,1.770,0.065000,3.660000,129.000000,0.05000,3.030000,AT,True,2020,3,1,NaN
1536,EDPD_2020_JAN_TC_1,Região Centro De Portugal,2020-01-19 00:00:00+00:00,VENTO INTENSIDADE EXCECIONAL,118.0,3364.0,0.010,0.000426,0.015498,0.632506,0.00026,0.010409,AT,True,2020,1,19,NaN
3416,EDPD_2016_FEV_TC_001,"Distritos De Aveiro, Braga, Castelo Branco, Co...",2016-02-14 00:00:00+00:00,VENTO INTENSIDADE EXCEPCIONAL,2400.0,359416.0,3.860,1070.000000,6.560000,284.000000,703.00000,4.410000,BT,True,2016,2,14,NaN
3632,EDPD_2015_NOV_TC_001,Distritos De Beja E Faro,2015-11-01 00:00:00+00:00,INUNDAÇÕES IMPREVISÍVEIS,2100.0,78716.0,0.747,79000.000000,0.847000,49.400000,86400.00000,0.966000,BT,True,2015,11,1,NaN
3704,EDPD_2015_OUT_TC_001,"Distritos De Leiria, Santarém E Lisboa",2015-10-17 00:00:00+00:00,VENTO INTENSIDADE EXCEPCIONAL,2100.0,301799.0,4.530,6450.000000,6.300000,301.000000,4500.00000,4.950000,BT,True,2015,10,17,NaN
